In [2]:
import pandas as pd
import os

In [3]:
# Đường dẫn thư mục chứa các file Excel
base_path = 'Data/Data raw'

In [4]:
# Đọc từng file Excel và gán vào các biến tương ứng
data1 = pd.read_excel(f'{base_path}/2020-Vietnam.xlsx')
data2 = pd.read_excel(f'{base_path}/2021-Vietnam.xlsx')
data3 = pd.read_excel(f'{base_path}/2022-Vietnam.xlsx')
data4 = pd.read_excel(f'{base_path}/2023-Vietnam.xlsx')
data5 = pd.read_excel(f'{base_path}/2024-Vietnam.xlsx')

In [5]:
# Kiểm tra danh sách các cột
print(data1.columns)

Index(['Mã', 'Tên công ty', 'Sàn', 'Ngành ICB - cấp 1', 'Ngành ICB - cấp 2',
       'Ngành ICB - cấp 3', 'Ngành ICB - cấp 4',
       'Quý\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n',
       'Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n',
       'Trạng thái kiểm toán\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n',
       ...
       'TM. Chi phí tài chính khác\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí sản xuất theo yếu tố\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí nguyên liệu, vật liệu\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí nhân công\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM.Chi phí khấu hao tài sản cố định\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí dịch vụ mua ngoài\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'TM. Chi phí khác bằng tiền\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\nĐơn vị: Triệu VND',
       'BCTCKH. Doanh thu 

In [6]:
# Lọc các cột có chữ "LCTT." ở đầu
columns_to_select = ["Mã"] + [col for col in data1.columns if col.startswith("LCTT.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n" in data1.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n")  # Thêm "Năm" vào vị trí thứ hai

# Nếu biến year_column đã được định nghĩa và tồn tại trong data1, thêm vào danh sách
if 'year_column' in globals() and year_column in data1.columns:
    columns_to_select.insert(1, year_column)

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data1_LCTT = data1[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data1_LCTT.columns = data1_LCTT.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2020\n", "", regex=False)
data1_LCTT.columns = data1_LCTT.columns.str.replace("Đơn vị: Triệu VND", "", regex=False)
data1_LCTT.columns = data1_LCTT.columns.str.replace("LCTT.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data1_LCTT.columns:
    data1_LCTT = data1_LCTT.drop(columns=["Quý"])

# Đổi tên cột chứa thông tin năm thành "Năm"
if 'year_column' in globals() and year_column in data1.columns:
    data1_LCTT = data1_LCTT.rename(columns={year_column: "Năm"})

# Thay thế các giá trị NaN hoặc trống bằng 0
data1_LCTT = data1_LCTT.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data1_LCTT.columns:
    data1_LCTT["Năm"] = data1_LCTT["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Hiển thị DataFrame sau khi xử lý
data1_LCTT.head()


,Mã,Năm,Lãi trước thuế,Khấu hao TSCĐ,Lãi/(lỗ) trước những thay đổi vốn lưu động,Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT),"Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác (TT)","Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác (TT)","Tiền chi cho vay, mua các công cụ nợ của đợn vị khác (TT)","Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác (TT)",...,Tiền thu được các khoản đi vay (TT),Tiền trả nợ gốc vay (TT),Tiền thanh toán vốn gốc đi thuê tài chính (TT),Cổ tức đã trả (TT),Lưu chuyển tiền tệ từ hoạt động tài chính (TT),Lưu chuyển tiền thuần trong kỳ (TT),Tiền và tương đương tiền đầu kỳ (TT),Ảnh hưởng của chênh lệch tỷ giá (TT),Tiền và tương đương tiền cuối kỳ (TT),Đầu tư vào doanh nghiệp khác
0,ACB,2020,0.000000e+00,0.000000e+00,9.688384e+12,1.087926e+13,-4.231320e+11,0.000000e+00,0.000000e+00,0.000000e+00,...,2.940000e+10,-1.700000e+11,0.0,0.000000e+00,-1.406000e+11,1.034316e+13,3.567891e+13,0.000000e+00,4.602207e+13,0.000000e+00
1,BCM,2020,2.486851e+12,2.380000e+11,2.251378e+12,2.548369e+12,-3.365953e+11,4.921206e+09,0.000000e+00,3.063014e+11,...,9.150313e+12,-8.836633e+12,0.0,-4.111564e+11,-9.747606e+10,1.358863e+12,4.167237e+11,0.000000e+00,1.775587e+12,-1.141116e+12
2,BID,2020,0.000000e+00,0.000000e+00,2.875826e+13,-6.791281e+13,-1.061633e+12,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-2.669096e+12,-2.669096e+12,-7.150116e+13,1.702724e+14,0.000000e+00,9.877124e+13,0.000000e+00
3,BVH,2020,1.953286e+12,2.069977e+11,1.163647e+13,1.179614e+13,-2.314335e+11,1.026440e+09,-8.790674e+13,7.168897e+13,...,1.745180e+13,-1.882909e+13,0.0,-6.215100e+11,-1.998801e+12,2.037359e+12,4.742602e+12,7.812019e+08,6.780742e+12,-2.953590e+10
4,CTG,2020,0.000000e+00,0.000000e+00,2.429897e+13,7.845636e+12,-7.154590e+11,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-1.881229e+12,-1.881229e+12,5.447022e+12,1.550467e+14,-3.186900e+11,1.601751e+14,2.399000e+10


In [7]:
# Lọc các cột có chữ "LCTT." ở đầu
columns_to_select = ["Mã"] + [col for col in data2.columns if col.startswith("LCTT.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2021\n" in data2.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2021\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data2_LCTT = data2[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data2_LCTT.columns = data2_LCTT.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2021\n", "", regex=False)
data2_LCTT.columns = data2_LCTT.columns.str.replace("Đơn vị: Triệu VND", "", regex=False)
data2_LCTT.columns = data2_LCTT.columns.str.replace("LCTT.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data2_LCTT.columns:
    data2_LCTT = data2_LCTT.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data2_LCTT = data2_LCTT.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên
if "Năm" in data2_LCTT.columns:
    data2_LCTT["Năm"] = data2_LCTT["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Hiển thị DataFrame sau khi xử lý
data2_LCTT.head()


,Mã,Năm,Lãi trước thuế,Khấu hao TSCĐ,Lãi/(lỗ) trước những thay đổi vốn lưu động,Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT),"Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác (TT)","Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác (TT)","Tiền chi cho vay, mua các công cụ nợ của đợn vị khác (TT)","Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác (TT)",...,Tiền thu được các khoản đi vay (TT),Tiền trả nợ gốc vay (TT),Tiền thanh toán vốn gốc đi thuê tài chính (TT),Cổ tức đã trả (TT),Lưu chuyển tiền tệ từ hoạt động tài chính (TT),Lưu chuyển tiền thuần trong kỳ (TT),Tiền và tương đương tiền đầu kỳ (TT),Ảnh hưởng của chênh lệch tỷ giá (TT),Tiền và tương đương tiền cuối kỳ (TT),Đầu tư vào doanh nghiệp khác
0,ACB,2021,0.000000e+00,0.000000e+00,1.232657e+13,3.992590e+13,-6.361000e+11,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-2.884000e+12,0.0,0.000000e+00,-2.884000e+12,3.657950e+13,4.602207e+13,0.000000e+00,8.260157e+13,0.000000e+00
1,BCM,2021,1.743748e+12,1.859899e+11,1.672982e+12,3.426676e+11,-2.583327e+10,8.198546e+09,-2.615062e+11,1.154349e+11,...,9.853676e+12,-7.080864e+12,0.0,-8.280000e+11,1.944812e+12,1.209163e+12,1.775587e+12,0.000000e+00,2.984749e+12,-1.005040e+12
2,BID,2021,0.000000e+00,0.000000e+00,4.359231e+13,6.889014e+13,-1.015514e+12,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-1.353760e+12,-1.353760e+12,6.655902e+13,9.877124e+13,0.000000e+00,1.653303e+14,-5.372500e+10
3,BVH,2021,2.398034e+12,2.263118e+11,1.370214e+13,1.232165e+13,-2.377686e+11,1.585989e+09,-9.478537e+13,7.217786e+13,...,4.806315e+13,-4.683680e+13,0.0,-6.874348e+11,5.389117e+11,-1.426133e+12,6.780742e+12,-5.170360e+08,5.354092e+12,-4.425976e+09
4,CTG,2021,0.000000e+00,0.000000e+00,3.061253e+13,5.401509e+12,-7.364080e+11,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-2.126700e+10,-2.126700e+10,4.732099e+12,1.601751e+14,-5.305890e+11,1.643766e+14,0.000000e+00


In [8]:
# Lọc các cột có chữ "LCTT." ở đầu
columns_to_select = ["Mã"] + [col for col in data3.columns if col.startswith("LCTT.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2022\n" in data3.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2022\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data3_LCTT = data3[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data3_LCTT.columns = data3_LCTT.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2022\n", "", regex=False)
data3_LCTT.columns = data3_LCTT.columns.str.replace("Đơn vị: Triệu VND", "", regex=False)
data3_LCTT.columns = data3_LCTT.columns.str.replace("LCTT.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data3_LCTT.columns:
    data3_LCTT = data3_LCTT.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data3_LCTT = data3_LCTT.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data3_LCTT.columns:
    data3_LCTT["Năm"] = data3_LCTT["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Hiển thị DataFrame sau khi xử lý
data3_LCTT.head()


,Mã,Năm,Lãi trước thuế,Khấu hao TSCĐ,Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT),"Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác (TT)","Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác (TT)","Tiền chi cho vay, mua các công cụ nợ của đợn vị khác (TT)","Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác (TT)",Lưu chuyển tiền tệ ròng từ hoạt động đầu tư (TT),...,Tiền thu được các khoản đi vay (TT),Tiền trả nợ gốc vay (TT),Tiền thanh toán vốn gốc đi thuê tài chính (TT),Cổ tức đã trả (TT),Lưu chuyển tiền tệ từ hoạt động tài chính (TT),Lưu chuyển tiền thuần trong kỳ (TT),Tiền và tương đương tiền đầu kỳ (TT),Ảnh hưởng của chênh lệch tỷ giá (TT),Tiền và tương đương tiền cuối kỳ (TT),Đầu tư vào doanh nghiệp khác
0,ACB,2022,0.000000e+00,0.000000e+00,2.127454e+13,-5.115130e+11,0.000000e+00,0.000000e+00,0.000000e+00,-3.658780e+11,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,2.090866e+13,8.260157e+13,0.000000e+00,1.035102e+14,-1.052000e+10
1,BCM,2022,1.894709e+12,1.897133e+11,5.038973e+11,-2.122377e+11,1.034425e+11,-2.224051e+11,1.379857e+09,-8.802413e+11,...,5.608259e+12,-6.421517e+12,0.0,-7.245000e+11,-1.537759e+12,-1.914103e+12,2.984749e+12,0.000000e+00,1.070647e+12,-7.437820e+11
2,BID,2022,0.000000e+00,0.000000e+00,1.325179e+14,-1.091733e+12,0.000000e+00,0.000000e+00,0.000000e+00,-9.178830e+11,...,0.000000e+00,0.000000e+00,0.0,-3.019300e+11,7.570950e+11,1.323571e+14,1.653303e+14,0.000000e+00,2.976874e+14,0.000000e+00
3,BVH,2022,2.010164e+12,2.118040e+11,9.101716e+12,-1.633784e+11,2.489491e+09,-1.579488e+14,1.289885e+14,-2.074397e+13,...,8.559162e+13,-7.482393e+13,0.0,-2.268459e+12,8.499231e+12,-3.143025e+12,5.354092e+12,-4.569950e+09,2.206498e+12,-2.500000e+08
4,CTG,2022,0.000000e+00,0.000000e+00,8.446373e+13,-6.236800e+11,0.000000e+00,0.000000e+00,0.000000e+00,-5.150810e+11,...,0.000000e+00,0.000000e+00,0.0,-3.844600e+12,-3.844600e+12,8.010405e+13,1.643766e+14,-1.824140e+11,2.442982e+14,0.000000e+00


In [9]:
# Lọc các cột có chữ "LCTT." ở đầu
columns_to_select = ["Mã"] + [col for col in data4.columns if col.startswith("LCTT.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2023\n" in data4.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2023\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data4_LCTT = data4[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data4_LCTT.columns = data4_LCTT.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2023\n", "", regex=False)
data4_LCTT.columns = data4_LCTT.columns.str.replace("Đơn vị: Tỷ VND", "", regex=False)
data4_LCTT.columns = data4_LCTT.columns.str.replace("LCTT.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data4_LCTT.columns:
    data4_LCTT = data4_LCTT.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data4_LCTT = data4_LCTT.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên (nếu cột "Năm" tồn tại)
if "Năm" in data4_LCTT.columns:
    data4_LCTT["Năm"] = data4_LCTT["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Lấy danh sách các cột cần chuyển đổi (ngoại trừ "Mã" và "Năm")
cols_to_convert = [col for col in data4_LCTT.columns if col not in ["Mã", "Năm"]]

# Nhân giá trị của các cột đó với 1000
data4_LCTT[cols_to_convert] = data4_LCTT[cols_to_convert].apply(lambda x: x * 1000)

# Hiển thị DataFrame sau khi xử lý
data4_LCTT.head()


,Mã,Năm,Lãi trước thuế,Khấu hao TSCĐ,Lãi/(lỗ) trước những thay đổi vốn lưu động,Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT),"Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác (TT)","Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác (TT)","Tiền chi cho vay, mua các công cụ nợ của đợn vị khác (TT)","Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác (TT)",...,Tiền thu được các khoản đi vay (TT),Tiền trả nợ gốc vay (TT),Tiền thanh toán vốn gốc đi thuê tài chính (TT),Cổ tức đã trả (TT),Lưu chuyển tiền tệ từ hoạt động tài chính (TT),Lưu chuyển tiền thuần trong kỳ (TT),Tiền và tương đương tiền đầu kỳ (TT),Ảnh hưởng của chênh lệch tỷ giá (TT),Tiền và tương đương tiền cuối kỳ (TT),Đầu tư vào doanh nghiệp khác
0,ACB,2023,0.000000e+00,0.000000,1.942198e+07,3.713620e+07,-1.276249e+06,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-3.377435e+06,-3.377435e+06,3.256151e+07,1.035102e+08,0.000000,1.360717e+08,0.000000
1,BCM,2023,2.697158e+06,267444.334094,2.908546e+06,-3.150641e+06,-1.147808e+05,1282.029931,3.501810e+05,0.000000e+00,...,9.983931e+06,-6.216115e+06,0.0,-8.280001e+05,2.939816e+06,2.616075e+05,1.070647e+06,0.000000,1.332254e+06,-859059.720000
2,BID,2023,0.000000e+00,0.000000,5.008039e+07,-7.101499e+07,-2.005077e+06,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-1.196940e+05,-1.196940e+05,-7.294838e+07,2.976874e+08,0.000000,2.247390e+08,0.000000
3,BVH,2023,2.236299e+06,203390.740109,1.061020e+07,8.476849e+06,-1.792236e+05,1056.681274,-1.639018e+08,1.479644e+08,...,7.833299e+07,-7.893651e+07,0.0,-7.341024e+05,-1.337622e+06,2.575844e+06,2.206498e+06,1172.235511,4.783514e+06,-559.065379
4,CTG,2023,0.000000e+00,0.000000,5.118523e+07,2.876835e+07,-8.041010e+05,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,2.806447e+07,2.442982e+08,-59079.000000,2.723036e+08,0.000000


In [10]:
# Lọc các cột có chữ "LCTT." ở đầu
columns_to_select = ["Mã"] + [col for col in data5.columns if col.startswith("LCTT.")]

# Nếu cột "Năm" tồn tại, thêm vào danh sách
if "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2024\n" in data5.columns:
    columns_to_select.insert(1, "Năm\nHợp nhất\nQuý: Hàng năm\nNăm: 2024\n")  # Thêm "Năm" vào vị trí thứ hai

# Tạo DataFrame mới chỉ chứa các cột cần thiết
data5_LCTT = data5[columns_to_select]

# Loại bỏ các ký tự không mong muốn khỏi tên cột
data5_LCTT.columns = data5_LCTT.columns.str.replace("\nHợp nhất\nQuý: Hàng năm\nNăm: 2024\n", "", regex=False)
data5_LCTT.columns = data5_LCTT.columns.str.replace("Đơn vị: Tỷ VND", "", regex=False)
data5_LCTT.columns = data5_LCTT.columns.str.replace("LCTT.", "", regex=False)

# Loại bỏ cột "Quý" nếu tồn tại
if "Quý" in data5_LCTT.columns:
    data5_LCTT = data5_LCTT.drop(columns=["Quý"])

# Thay thế các giá trị NaN hoặc trống bằng 0
data5_LCTT = data5_LCTT.fillna(0)

# Chuyển giá trị trong cột "Năm" thành số nguyên
if "Năm" in data5_LCTT.columns:
    data5_LCTT["Năm"] = data5_LCTT["Năm"].astype(str).str.extract(r'(\d+)').astype(int)

# Lấy danh sách các cột cần chuyển đổi (ngoại trừ "Mã" và "Năm")
cols_to_convert = [col for col in data5_LCTT.columns if col not in ["Mã", "Năm"]]

# Nhân giá trị của các cột đó với 1000
data5_LCTT[cols_to_convert] = data5_LCTT[cols_to_convert].apply(lambda x: x * 1000)

# Hiển thị DataFrame sau khi xử lý
data5_LCTT.head()


,Mã,Năm,Lãi trước thuế,Khấu hao TSCĐ,Lãi/(lỗ) trước những thay đổi vốn lưu động,Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT),"Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác (TT)","Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác (TT)","Tiền chi cho vay, mua các công cụ nợ của đợn vị khác (TT)","Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác (TT)",...,Tiền thu được các khoản đi vay (TT),Tiền trả nợ gốc vay (TT),Tiền thanh toán vốn gốc đi thuê tài chính (TT),Cổ tức đã trả (TT),Lưu chuyển tiền tệ từ hoạt động tài chính (TT),Lưu chuyển tiền thuần trong kỳ (TT),Tiền và tương đương tiền đầu kỳ (TT),Ảnh hưởng của chênh lệch tỷ giá (TT),Tiền và tương đương tiền cuối kỳ (TT),Đầu tư vào doanh nghiệp khác
0,ACB,2024,0.000000e+00,0.000000,1.670989e+07,8.370232e+06,-9.197400e+05,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-3.884051e+06,-3.884051e+06,3.752870e+06,1.360717e+08,0.000000,1.398246e+08,0.000000e+00
1,BCM,2024,2.451490e+06,262068.871772,2.867864e+06,-7.773522e+05,-2.606734e+05,5957.317290,-2.318986e+05,2.430000e+04,...,1.473196e+07,-1.088948e+07,0.0,-1.028798e+06,2.813678e+06,8.778858e+05,1.332254e+06,23.607485,2.210164e+06,-1.286292e+06
2,BID,2024,0.000000e+00,0.000000,3.983713e+07,1.010801e+08,-1.244829e+06,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,-9.446000e+04,-9.446000e+04,9.985821e+07,2.247390e+08,0.000000,3.245972e+08,0.000000e+00
3,BVH,2024,2.620410e+06,360708.436055,1.007561e+07,9.225095e+06,-1.326894e+05,750.785454,-1.788878e+08,1.439192e+08,...,6.802677e+07,-5.736592e+07,0.0,-7.670814e+05,9.893761e+06,-3.323657e+06,4.783514e+06,720.102279,1.460577e+06,-2.453085e+02
4,CTG,2024,0.000000e+00,0.000000,4.954669e+07,1.021464e+08,-1.319839e+06,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.537000e+03,1.008612e+08,2.723036e+08,154749.000000,3.733196e+08,0.000000e+00


In [ ]:
# Đường dẫn thư mục xuất file
output_dir = r"Data\Data cleaned"
os.makedirs(output_dir, exist_ok=True)

# Tạo đường dẫn file Excel mới với tên "data_LCTT.xlsx"
output_file = os.path.join(output_dir, "data_LCTT.xlsx")

# Xuất các DataFrame data1_LCTT đến data5_LCTT vào file Excel với các sheet riêng biệt
with pd.ExcelWriter(output_file) as writer:
    data1_LCTT.to_excel(writer, sheet_name="data1_LCTT", index=False)
    data2_LCTT.to_excel(writer, sheet_name="data2_LCTT", index=False)
    data3_LCTT.to_excel(writer, sheet_name="data3_LCTT", index=False)
    data4_LCTT.to_excel(writer, sheet_name="data4_LCTT", index=False)
    data5_LCTT.to_excel(writer, sheet_name="data5_LCTT", index=False)

print(f"File Excel đã được lưu tại: {output_file}")


File Excel đã được lưu tại: Data\Data cleaned\data_KQKD.xlsx
